In [2]:
!pip install seaborn

In [3]:
import pandas as pd # working with tabular data
import pickle # for machine learning models
import seaborn as sns # visualization
import matplotlib.pyplot as plt # visualization

from sklearn.feature_extraction import DictVectorizer # Machine Learning
from sklearn.linear_model import LinearRegression # Machine Learning
from sklearn.linear_model import Lasso # Regularization
from sklearn.linear_model import Ridge # Regularization

from sklearn.metrics import mean_squared_error # Loss Function

In [4]:
import mlflow

# to hook up with MLFlow UI
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("mlops_nyc_taxi") # choose a name for your experiment

2025/05/15 22:23:17 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/05/15 22:23:17 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

<Experiment: artifact_location='file:///c:/Users/usEr/Documents/GitHub/cpe393-mlflow/mlruns/1', creation_time=1747322598424, experiment_id='1', last_update_time=1747322598424, lifecycle_stage='active', name='mlops_nyc_taxi', tags={}>

In [5]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [8]:
df_train = read_dataframe('yellow_tripdata_2022-01.parquet')
df_val = read_dataframe('yellow_tripdata_2022-02.parquet')

In [9]:
len(df_train), len(df_val)

(2421440, 2918187)

In [10]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [11]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred)

30.58722882751861

In [14]:
with open('./lin_reg.bin', 'wb') as f_out: # wb means write binary mlops-zoomcamp/week_1/models
    try:
        # Pickle both the dictionary vectorizer and the linear regression model
        pickle.dump((dv, lr), f_out)
        print("Model successfully pickled.")
    except Exception as e:
        print("Error occurred while pickling the model:", str(e))

Model successfully pickled.


In [15]:
!pip install xgboost
!pip install hyperopt

In [16]:
# import required modules
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials # some methods to optimize hyperparameters
from hyperopt.pyll import scope

In [17]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [18]:
# Objective function - set the parameters for this specific run
def objective(params): 
    
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params) 
        booster = xgb.train(
            params=params,
            dtrain=train, # model trained on training set
            num_boost_round=3, # restricted due to time constraints - a value of 1000 iterations is common
            evals=[(valid, 'validation')], # model evaluated on validation set
            early_stopping_rounds=3 # if no improvements after 3 iterations, stop running # restricted, time constraints
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [19]:
# Set the range of the hyperparameter optimization search
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)), # tree depth 4 to 100. Returns float, so convert to integer
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # range exp(-3), exp(0) which is (0.049787, 1.0)
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1), # range exp(-5), exp(-1) which is (0.006738, 0.367879)
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1), # range exp(-6), exp(-1) which is (0.002479, 0.367879)
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3), # range exp(-1), exp(3) which is (0.367879, 20.085537)
    'objective': 'reg:linear',
    'seed': 42
}


best_result = fmin( # imported above
    fn=objective,
    space=search_space, # as defined above
    algo=tpe.suggest, # tpe is the algorithm used for optimization
    max_evals=3, #restricted, time constraints
    trials=Trials()
)

  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

C:\Users\usEr\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [22:24:11] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.27705                          
[1]	validation-rmse:8.88199                          
[2]	validation-rmse:8.52083                          
 33%|███▎      | 1/3 [01:14<02:28, 74.37s/trial, best loss: 72.60446541415935]

C:\Users\usEr\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [22:25:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:5.52689                                                   
[1]	validation-rmse:5.13447                                                   
[2]	validation-rmse:5.07826                                                   
 67%|██████▋   | 2/3 [01:56<00:55, 55.58s/trial, best loss: 25.78869564475516]

C:\Users\usEr\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [22:26:08] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:8.91785                                                   
[1]	validation-rmse:8.25293                                                   
[2]	validation-rmse:7.69675                                                   
100%|██████████| 3/3 [02:25<00:00, 48.44s/trial, best loss: 25.78869564475516]


In [20]:
mlflow.xgboost.autolog(disable=True) # MLflow will not store parameters automatically - these will have to be requested

In [21]:
#best params

params = {
    'learning_rate': 0.33500504398153885,
    'max_depth': 74,
    'min_child_weight': 3.4593376740383635,
    'objective': 'reg:linear',
    'reg_alpha': 0.02439429006088758,
    'reg_lambda': 0.0029162481576915956,
    'seed': 42   
}

In [22]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.4434065752589766,
        'max_depth': 81,
        'min_child_weight': 10.423237853746643,
        'objective': 'reg:linear',
        'reg_alpha': 0.2630756846813668,
        'reg_lambda': 0.1220536223877784,
        'seed': 42    
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=params,
        dtrain=train, # model trained on training set
        num_boost_round=3, # restricted due to time constraints - a value of 1000 iterations is common
        evals=[(valid, 'validation')], # model evaluated on validation set
        early_stopping_rounds=3 # if no improvements after 3 iterations, stop running # restricted, time constraints
        )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("preprocessor.b", "wb") as f_out: # save pre-processing as a model
        pickle.dump(dv, f_out)
        
    mlflow.log_artifact("preprocessor.b", artifact_path="preprocessor") # we can isolate the pre-processing from raw data
    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow") 

C:\Users\usEr\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [22:26:37] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:227: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:7.64848
[1]	validation-rmse:6.47024
[2]	validation-rmse:5.82476


C:\Users\usEr\AppData\Roaming\Python\Python312\site-packages\xgboost\core.py:158: UserWarning: [22:28:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0c55ff5f71b100e98-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/05/15 22:28:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


In [34]:
logged_model = 'mlruns/639091011841886520/9c6de352d91d4d049eb8b24d5b480b0a/artifacts/models'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

In [35]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: models
  flavor: mlflow.sklearn
  run_id: 9c6de352d91d4d049eb8b24d5b480b0a

In [36]:
sklearn_model = mlflow.sklearn.load_model(logged_model)

In [37]:
sklearn_model

LogisticRegression(C=0.1, random_state=42)

In [38]:
# check the first 10
y_pred[:10]

array([14.190828, 18.68855 , 26.193169, 23.354773, 26.193169, 13.043591,
       21.33266 ,  6.970524, 15.222571, 15.575311], dtype=float32)